In [1]:
# !pip install mygene
# !pip install biothings_client

In [1]:
from biothings_client import get_client
mg = get_client("gene")

In [23]:
# キーワードの検索結果件数と一覧

# mg.query(keyword)

# {'took': 12,
#  'total': 2307,
#  'max_score': 394.02466,
#  'hits': [{'_id': '10845',
#    '_score': 394.02466,
#    'entrezgene': '10845',
#    'name': 'caseinolytic mitochondrial matrix peptidase chaperone subunit X',
#    'symbol': 'CLPX',
#    'taxid': 9606},
#   {'_id': '270166',.....

# gene_infoの各項目

In [25]:
gene_info = mg.getgene("ENSG00000141570")

In [9]:
for key,value in gene_info.items():
    print(key,":",type(value))

HGNC : <class 'str'>
MIM : <class 'str'>
_id : <class 'str'>
_version : <class 'int'>
accession : <class 'dict'>
alias : <class 'list'>
ensembl : <class 'dict'>
entrezgene : <class 'str'>
exac : <class 'dict'>
exons : <class 'list'>
exons_hg19 : <class 'list'>
generif : <class 'list'>
genomic_pos : <class 'dict'>
genomic_pos_hg19 : <class 'list'>
go : <class 'dict'>
homologene : <class 'dict'>
interpro : <class 'list'>
ipi : <class 'list'>
map_location : <class 'str'>
name : <class 'str'>
other_names : <class 'list'>
pantherdb : <class 'dict'>
pathway : <class 'dict'>
pdb : <class 'list'>
pfam : <class 'list'>
pharmgkb : <class 'str'>
pharos : <class 'dict'>
prosite : <class 'str'>
reagent : <class 'dict'>
refseq : <class 'dict'>
reporter : <class 'dict'>
retired : <class 'list'>
symbol : <class 'str'>
taxid : <class 'int'>
type_of_gene : <class 'str'>
umls : <class 'dict'>
unigene : <class 'str'>
uniprot : <class 'dict'>
wikipedia : <class 'dict'>


In [30]:
gene_info["ensembl"]
# {'gene': 'ENSG00000141570',
#  'protein': ['ENSP00000269385', 'ENSP00000405058', 'ENSP00000408753'],
#  'transcript': ['ENST00000269385',....],
#  'translation': [{'protein': 'ENSP00000269385', 'rna': 'ENST00000269385'},....],
#  'type_of_gene': 'protein_coding'}
gene_info["ensembl"]["gene"]
# 遺伝子に関する情報
gene_info["ensembl"]["protein"]
# 生成されるタンパク質種類

gene_info["entrezgene"]
#人間とか犬とか生物のカテゴリを表すID

gene_info["pdb"]
#PDB IDは重複しない４文字のコードで、タンパク質
#データバンクの各エントリーに割り振られます。
#最初の文字は必ず1から9までの数字で、残りの３文字は
#アルファベットまたは数字が使われます。

gene_info["generif"]
# 文献と出現テキストのリスト
# [{'pubmed': 17332741,'text': 'CBX8 is an essenti...'}]

gene_info["name"]
#正式名称

gene_info["other_names"]
# 正式名称表記ゆれ

gene_info["pdb"]
#タンパク質固有ID
# ['2N4Q', '3I91', '5EQ0']

['2N4Q', '3I91', '5EQ0']

# データの読み込み

dict

In [3]:
import pandas as pd
data = pd.read_csv('data_summary/PROT/lll_bioc_prot.csv').values.tolist()

In [4]:
import copy
new_data = copy.deepcopy(data)

In [5]:
new_data[0:15]

[['ClpX', 6],
 ['ComK', 10],
 ['CsbB', 2],
 ['CtsR', 6],
 ['DnaK', 8],
 ['E sigma E', 6],
 ['EsigmaF', 8],
 ['FlgM', 13],
 ['FtsZ', 25],
 ['GerE', 210],
 ['KatX', 8],
 ['KinC', 2],
 ['KinD', 2],
 ['PBP4*', 7],
 ['PhoP~P', 3]]

In [62]:
# import urllib

In [107]:
label = ["protein","appearances","hits","max_score_hits"]

In [35]:
from biothings_client import get_client
import copy
import json
import pandas as pd
import csv
#当然ながらネット接続時でないとエラーになる

# 生成先のディレクトリは手動作成
def get_Query(filename):
    mg = get_client("gene")
    data = pd.read_csv('data_summary/PROT/'+filename+'_prot.csv').values.tolist()
    new_data = copy.deepcopy(data)
    
    for i in range(len(new_data)):
        keyword = new_data[i][0]
        max_score_hit = []

        try:
            max_score_hit = []
            result = mg.query(keyword)
            wf = open('data_query/QUERY/all/'+filename+'/'+str(i)+'_'+keyword+'.json','w',encoding='UTF-8',newline='\n')
            wf.write(json.dumps(result))
            wf.close()

            search_results = result["total"]
            max_score = result["max_score"]

            if search_results != 0:
                wf = open('data_query/QUERY/best_score/'+filename+'/'+str(i)+'_'+keyword+'.json','w',encoding='UTF-8',newline='\n')
                for hit in result["hits"]:
                    if hit["_score"] == max_score:
                        wf.write(json.dumps(hit))
                        wf.write("\n")
                        max_score_hit.append(hit)
                wf.close()
            print(i,"/",len(new_data)-1,"   ",keyword,"=>",search_results,"(max_hits",len(max_score_hit),")")

            

        except:
            print(i,"/",len(new_data),"   HTTPError")
            search_results = -1

        new_data[i].append(search_results)
        new_data[i].append(len(max_score_hit))

    label = ["protein","appearances","hits","max_score_hits"]
    new_data.insert(0, label)
    with open("data_query/PROT/"+filename+"_result.csv", "w") as f:
        writer = csv.writer(f, lineterminator='\n')#ここ大事
        writer.writerows(new_data)

In [36]:
import os
files = os.listdir("data_original")

val_name = []
for a_file in files:
    with open("data_original/"+a_file, 'r') as fp:
        dynmc_name = a_file.replace(".xml","")
        val_name.append(dynmc_name)

In [37]:
val_name

['aimed_bioc', 'bioinfer_bioc', 'hprd50_bioc', 'iepa_bioc', 'lll_bioc']

In [40]:
for filename in val_name[:1]:
    print(filename)
    get_Query(filename)

aimed_bioc
0 / 1136     .CDC37 => 1670 (max_hits 10 )
1 / 1136     120-kDa phosphoprotein => 0 (max_hits 0 )
2 / 1136     14-3-3 beta => 1413 (max_hits 1 )
3 / 1136     14-3-3 zeta => 1710 (max_hits 2 )
4 / 1136     5 E1B => 1040 (max_hits 10 )
5 / 1136     53BP2 => 2 (max_hits 1 )
6 / 1136     56Lyn => 0 (max_hits 0 )
7 / 1136     80K-H => 448 (max_hits 1 )
8 / 1136     87K postsynaptic protein => 0 (max_hits 0 )
9 / 1136     A beta => 23648 (max_hits 1 )
10 / 1136     A kinase anchoring protein => 5679 (max_hits 1 )
11 / 1136     AADC => 65 (max_hits 10 )
12 / 1136     ABL-SH3 => 22 (max_hits 1 )
13 / 1136     ABP-280 => 7 (max_hits 1 )
14 / 1136     ADA => 974 (max_hits 1 )
15 / 1136     ADA-1 => 27 (max_hits 2 )
16 / 1136     AKAP75 => 2 (max_hits 1 )
17 / 1136     AKAP79 => 2 (max_hits 1 )
18 / 1136     AP-1 => 21456 (max_hits 7 )
19 / 1136     AP1 => 217 (max_hits 10 )
20 / 1136     APO-1 => 280 (max_hits 1 )
21 / 1136     AT mutated => 31 (max_hits 1 )
22 / 1136     ATF => 6673 

In [108]:
new_data.insert(0, label)

In [109]:
new_data

[['protein', 'appearances', 'hits', 'max_score_hits'],
 ['ClpX', 6, 2319, 1],
 ['ComK', 10, 166, 3],
 ['CsbB', 2, 4, 3],
 ['CtsR', 6, 226, 1],
 ['DnaK', 8, 2136, 10],
 ['E sigma E', 6, 1007, 10],
 ['EsigmaF', 8, 0, 0],
 ['FlgM', 13, 514, 10],
 ['FtsZ', 25, 2955, 10],
 ['GerE', 210, 28, 10],
 ['KatX', 8, 8, 7],
 ['KinC', 2, 4, 3],
 ['KinD', 2, 212, 2],
 ['PBP4*', 7, 88, 10],
 ['PhoP~P', 3, -1, 0],
 ['RacX', 7, 38, 1],
 ['RbsW', 8, 0, 0],
 ['RocR', 4, 17, 2],
 ['SigE', 6, 523, 10],
 ['SigK', 12, 146, 10],
 ['SigL', 5, 11, 4],
 ['Spo0', 1, 0, 0],
 ['Spo0A', 13, 174, 10],
 ['Spo0A~P', 2, -1, 0],
 ['SpoIIAA', 23, 86, 10],
 ['SpoIIAA-P', 12, 0, 0],
 ['SpoIIAB', 14, 58, 10],
 ['SpoIIE', 75, 1079, 10],
 ['SpoIIID', 60, 123, 10],
 ['YdjP', 7, 11, 4],
 ['YfhM', 7, 124, 7],
 ['YfhP', 6, 4, 2],
 ['YknXYZ', 7, 0, 0],
 ['YteI', 7, 0, 0],
 ['YtxH', 3, 545, 1],
 ['YuaG', 7, 0, 0],
 ['YvyD', 3, 2, 1],
 ['ald', 9, 1145, 10],
 ['araE', 1, 20, 10],
 ['bkd', 1, 40, 1],
 ['bmrUR', 7, 0, 0],
 ['clpC', 12, 53

In [110]:
import csv
filename = "lll_bioc_result.csv"
with open("data_query/PROT/"+filename, "w") as f:
    writer = csv.writer(f, lineterminator='\n')#ここ大事
    writer.writerows(new_data)

In [68]:
new_data_row_2 = [row[2] for row in new_data]

In [72]:
new_data_row_2

[2319,
 166,
 4,
 226,
 2136,
 1007,
 0,
 514,
 2955,
 28,
 8,
 4,
 212,
 88,
 -1,
 38,
 0,
 17,
 523,
 146,
 11,
 0,
 174,
 -1,
 86,
 0,
 58,
 1079,
 123,
 11,
 124,
 4,
 0,
 0,
 545,
 0,
 2,
 1145,
 20,
 40,
 0,
 539,
 817,
 258,
 3,
 3,
 25,
 2,
 2,
 2,
 514,
 1718,
 28,
 171,
 41,
 8,
 142,
 1117,
 10,
 3,
 220,
 447,
 523,
 146,
 40,
 90,
 1108,
 1047,
 170,
 368,
 278,
 64,
 44,
 5476,
 113,
 142,
 281,
 87,
 26,
 7,
 7,
 1,
 0,
 0,
 2,
 1,
 0,
 0,
 266,
 0,
 0,
 174,
 0,
 0,
 121,
 223,
 2,
 1,
 1,
 12,
 24,
 512,
 458,
 167,
 121,
 36,
 1,
 2,
 2,
 3,
 4,
 1,
 17,
 2,
 1,
 5,
 1,
 0,
 0,
 2,
 0]

In [83]:
keyword = "ykvP"
mg.query(keyword)

{'took': 4,
 'total': 1,
 'max_score': 426.04236,
 'hits': [{'_id': '939278',
   '_score': 426.04236,
   'entrezgene': '939278',
   'name': 'spore protein (HGT island)',
   'symbol': 'ykvP',
   'taxid': 224308}]}

In [74]:
search_results

1

In [93]:
keyword = "YfhP"
mg.query(keyword)

{'took': 6,
 'total': 4,
 'max_score': 413.27167,
 'hits': [{'_id': '936190',
   '_score': 413.27167,
   'entrezgene': '936190',
   'name': 'putative membrane hydrolase',
   'symbol': 'yfhP',
   'taxid': 224308},
  {'_id': '1254066',
   '_score': 413.27167,
   'entrezgene': '1254066',
   'name': 'Fe-S cluster assembly transcriptional regulator IscR',
   'symbol': 'yfhP',
   'taxid': 99287},
  {'_id': '107885630',
   '_score': 54.268234,
   'entrezgene': '107885630',
   'name': 'uncharacterized protein YfhP-like',
   'symbol': 'LOC107885630',
   'taxid': 7029},
  {'_id': '945279',
   '_score': 8.232498,
   'entrezgene': '945279',
   'name': 'DNA-binding transcriptional dual regulator IscR',
   'symbol': 'iscR',
   'taxid': 511145}]}

In [87]:
t